In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import yaml
import json
from yaml import Loader

In [11]:
with open("././selfplay_params.yaml") as f:
    params = yaml.load(f, Loader=Loader)
print(json.dumps(params, indent=2))

{
  "board": {
    "size": 15,
    "initialize": {
      "type": "random",
      "stones": 4,
      "left": 5,
      "right": 9,
      "upper": 5,
      "lower": 9
    }
  },
  "players": [
    {
      "mcts": {
        "cpuct": 4.0,
        "num_simulations": 100,
        "temperature": 1.0,
        "advice_cutoff": 0.5
      },
      "model": {
        "type": "HEURISTIC"
      }
    },
    {
      "mcts": {
        "cpuct": 4.0,
        "num_simulations": 100,
        "temperature": 1.0,
        "advice_cutoff": 0.5
      },
      "model": {
        "type": "HEURISTIC"
      }
    }
  ],
  "process": {
    "workers": {
      "trajectories": 8,
      "policy": 4,
      "selfplay": 8
    }
  }
}


dict